<a href="https://colab.research.google.com/github/bhattacharyyad/Debnath_Code/blob/master/FAISS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [131]:
!uv pip install faiss-cpu
!uv pip install sentence-transformers

Using Python 3.12.12 environment at: /usr
Audited 1 package in 90ms
Using Python 3.12.12 environment at: /usr
Audited 1 package in 140ms


In [132]:
import pandas as pd
data = [['Where are your headquarters located?', 'location'],
['Throw my cellphone in the water', 'random'],
['Network Access Control?', 'networking'],
['Address', 'location']]
df = pd.DataFrame(data, columns = ['text', 'category'])

In [133]:
from sentence_transformers import SentenceTransformer
text = df['text']
encoder = SentenceTransformer("paraphrase-mpnet-base-v2")
vectors = encoder.encode(text)

In [134]:
import faiss

vector_dimension = vectors.shape[1]
index = faiss.IndexFlatL2(vector_dimension)
faiss.normalize_L2(vectors)
index.add(vectors)

In [135]:
import numpy as np

search_text = 'where is your office?'
search_vector = encoder.encode(search_text)
_vector = np.array([search_vector])
faiss.normalize_L2(_vector)

In [136]:
k = index.ntotal
distances, ann = index.search(_vector, k=k)

In [137]:
results = pd.DataFrame({'distances': distances[0], 'ann': ann[0]})
results

,distances,ann
0,0.584873,0
1,1.175950,3
2,1.644265,2
3,1.919767,1


In [138]:
df

,text,category
0,Where are your headquarters located?,location
1,Throw my cellphone in the water,random
2,Network Access Control?,networking
3,Address,location


In [139]:
merge = pd.merge(results, df, left_on='ann', right_index=True)

In [140]:
merge

,distances,ann,text,category
0,0.584873,0,Where are your headquarters located?,location
1,1.175950,3,Address,location
2,1.644265,2,Network Access Control?,networking
3,1.919767,1,Throw my cellphone in the water,random


In [141]:
labels  = df['category']
category = labels[ann[0][0]]

In [142]:
labels

,category
0,location
1,random
2,networking
3,location


In [143]:
category

'location'

In [144]:
import faiss
import numpy as np

# Generate random data
data = np.random.rand(1000, 128).astype('float32')

# Create an index
index = faiss.IndexFlatL2(128)

# Add data to the index
index.add(data)

# Perform a similarity search
query_vector = np.random.rand(1, 128).astype('float32')
k = 5  # Number of nearest neighbors to retrieve
distances, indices = index.search(query_vector, k)

# Print the results
print("Nearest neighbors:")
for i in range(k):
    print(f"Neighbor {i+1}: Index {indices[0][i]}, Distance {distances[0][i]}")

Nearest neighbors:
Neighbor 1: Index 857, Distance 15.838767051696777
Neighbor 2: Index 540, Distance 16.027814865112305
Neighbor 3: Index 743, Distance 16.092527389526367
Neighbor 4: Index 216, Distance 16.584854125976562
Neighbor 5: Index 393, Distance 17.01280975341797


In [146]:
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
import os

# --- Configuration ---
FILE_PATH = 'document.txt'
INDEX_PATH = 'faiss_index.bin'
MODEL_NAME = 'sentence-transformers/all-MiniLM-L6-v2'
K_NEIGHBORS = 3

# 1. Load data from the file
def load_text_data(filepath):
    with open(filepath, 'r') as f:
        documents = f.readlines()
    # Strip whitespace/newlines
    documents = [doc.strip() for doc in documents if doc.strip()]
    return documents

documents = load_text_data(FILE_PATH)
print(f"Loaded {len(documents)} documents.")

# 2. Generate vector embeddings
print(f"Loading model: {MODEL_NAME}")
model = SentenceTransformer(MODEL_NAME)
document_embeddings = model.encode(documents)
dim = document_embeddings.shape[1]
print(f"Embeddings shape: {document_embeddings.shape}, Dimension: {dim}")

# Ensure embeddings are float32 as required by FAISS
document_embeddings = document_embeddings.astype('float32')

# 3. Create, populate, and save the FAISS index
# We use IndexFlatL2 for brute-force L2 (Euclidean) distance search
index = faiss.IndexFlatL2(dim)
index.add(document_embeddings)
faiss.write_index(index, INDEX_PATH)
print(f"FAISS index created and saved to {INDEX_PATH}.")
print(f"Total vectors in index: {index.ntotal}")

# 4. Load the index from the file (simulating a separate process/session)
print(f"\nLoading index from {INDEX_PATH} for search...")
loaded_index = faiss.read_index(INDEX_PATH)
print(f"Loaded index total vectors: {loaded_index.ntotal}")

# 5. Perform a search
query_text = "Delhi"
query_vector = model.encode([query_text]).astype('float32')

print(f"\nSearching for: '{query_text}'")
distances, indices = loaded_index.search(query_vector, K_NEIGHBORS)

# 6. Display results
print(f"Found {K_NEIGHBORS} nearest neighbors:")
for i in range(K_NEIGHBORS):
    idx = indices[0][i]
    dist = distances[0][i]
    print(f"* Neighbor {i+1} (Distance: {dist:.4f}): {documents[idx]}")


Loaded 5 documents.
Loading model: sentence-transformers/all-MiniLM-L6-v2
Embeddings shape: (5, 384), Dimension: 384
FAISS index created and saved to faiss_index.bin.
Total vectors in index: 5

Loading index from faiss_index.bin for search...
Loaded index total vectors: 5

Searching for: 'Delhi'
Found 3 nearest neighbors:
* Neighbor 1 (Distance: 0.7496): The Capital of India is New Delhi.
* Neighbor 2 (Distance: 1.2196): The Capital of Pakistan is Islamabad.
* Neighbor 3 (Distance: 1.3022): The Capital of Bangladesh is Dhaka.
